In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime
import os
import numpy as np
import re
import statistics
from classes import Speed, Time, Direction

In [2]:
WEATHER_DIR = "./weather/"
CITY_SUFFIXES = ["_melbourne", "_sydney", "_adelaide", "_brisbane"]
CITY_DIRS = [WEATHER_DIR + "weather" + suffix + ".csv" for suffix in CITY_SUFFIXES]

#loads city weather dfs
mel, syd, adl, brs = [pd.read_csv(city_dir) for city_dir in CITY_DIRS]


#loads price df
price_demand = pd.read_csv(WEATHER_DIR + 'price_demand_data.csv')


In [3]:

#these dont have number values . N, NW, SSE etc...
direction_columns = ["Direction of maximum wind gust ", "9am wind direction", "3pm wind direction"]
speed_columns = ["Speed of maximum wind gust (km/h)", "9am wind speed (km/h)", "3pm wind speed (km/h)"]


In [4]:
directions = [Direction(mel[series]).process().sort_values() for series in direction_columns]
speeds = [Speed(mel[series]).process() for series in speed_columns]

In [5]:
Time(mel["Date"]).days_after_min()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
num_rows = 5
num_cols = 2
weather_df = mel
max_plot_per_fig = int(20 / (num_rows * num_cols))

fig, ax = plt.subplots(num_rows, num_cols, figsize=(16,16))

time_differences = get_time_differences(weather_df["Date"])

#excludes date field
fields = list(weather_df.columns)[1:]
for i in range(0,num_rows):
    for j in range(0,num_cols):
        field_i_lower = (num_cols * i + j) * max_plot_per_fig
        field_i_upper = field_i_lower + max_plot_per_fig
        print("upper: ", field_i_upper)
        print("lower: ", field_i_lower)
        print(fields[field_i_lower : field_i_upper])
        [ax[i, j].plot(time_differences, weather_df[field]) for field in fields[field_i_lower : field_i_upper] if (field not in qualitative_columns and field not in float_columns)]
        ax[i, j].legend(fields[field_i_lower : field_i_upper])

#[(ax[0].plot(time_differences, weather_df[field])) for field in fields[:max_plot_per_fig]]
#ax[0].legend(fields[:4])

In [ ]:
fig2, ax2 = plt.subplots()
ax2.plot(get_time_differences(mel["Date"]), mel["Minimum temperature (°C)"])
ax2.plot(get_time_differences(mel["Date"]), mel["Maximum temperature (°C)"])
ax2.legend(["Minimum temperature","maximum temperature"])

In [ ]:
def get_outlier1(df, col):
    """ Takes a dataframe and a colum name
        Returns the index of the ones that are indentified as outliers
        Note: outliers identified based on the change in values
        therefore only work for values that are considered more or less continuous
        (data that are dependent on what happen the day before)
        e.g. temperature anti-example rainfall"""

    distance_lst=[df[col][i] - df[col][i-1] for i in range(1, len(df[col]))]
    Q1=np.nanpercentile(distance_lst, 25)
    Q3=np.nanpercentile(distance_lst, 75)
    interquartile_range=1.5*(Q3-Q1)
    upper_bound=Q3+interquartile_range
    lower_bound=Q1-interquartile_range
    outliers_index=[]
    for i in range(0,len(distance_lst)):
        if distance_lst[i]>upper_bound or distance_lst[i]<lower_bound:
            outliers_index.append(i+1)  
    return outliers_index

In [ ]:
price_demand.tail()
price_demand.sort_values(by=["REGION","SETTLEMENTDATE"], inplace=True)
price_demand.reset_index(inplace=True,drop=True)
price_demand.groupby(['REGION']).count()

In [ ]:
'''create a new dataframe where the total demand for each day is the average of all 
   and pricesurge is true if it's true for any time of the day'''

'''saved to a file named price_demand_sorte
   0-59999 NSW 60000-11999 QLD  12000-17999 SA    18000-23999 VIC'''

settlementdate_format=r'\d{4}/\d{2}/\d{2}'
settlementdate=list()
for dates in price_demand['SETTLEMENTDATE']:
    match=re.search(settlementdate_format, dates)
    if match.group(0) not in settlementdate:
        settlementdate.append(match.group(0))
price_demand_dated =pd.DataFrame(columns=price_demand.columns)
start_row=0
while start_row<239999:
    end_row=start_row
    BOOL=price_demand['PRICESURGE'][start_row]
    while re.search(settlementdate_format, price_demand['SETTLEMENTDATE'][end_row]).group(0)==re.search(settlementdate_format, price_demand['SETTLEMENTDATE'][start_row]).group(0) and price_demand['REGION'][start_row]==price_demand['REGION'][end_row]:
        if price_demand['PRICESURGE'][end_row]:
            BOOL=True
        end_row+=1
    price_demand_dated=price_demand_dated.append({"REGION": price_demand['REGION'][start_row],
                               "SETTLEMENTDATE": re.search(settlementdate_format, price_demand['SETTLEMENTDATE'][start_row]).group(0),
                               "TOTALDEMAND": round(statistics.mean(price_demand['TOTALDEMAND'][start_row:end_row]),2),
                               "PRICESURGE": BOOL}, ignore_index=True)
    start_row=end_row+1
